In [2]:
import spacy
from spacy import displacy
import requests
from bs4 import BeautifulSoup
import wikipediaapi
from wikipediaapi.wikipedia import WikipediaPage, WikipediaPageSection
from uuid import uuid4
import hashlib
from itertools import zip_longest

In [30]:
nlp = spacy.load('en')

In [17]:
s = """hello. Between 1507 and 1511 Dürer worked on some of his most celebrated paintings: 
Adam and Eve (1507), The Martyrdom of the Ten Thousand (1508, for Frederick of Saxony), 
Virgin with the Iris (1508), the altarpiece Assumption of the Virgin (1509, for Jacob Heller of Frankfurt), 
and Adoration of the Trinity (1511, for Matthaeus Landauer)."""

In [32]:
doc = nlp(s)

In [45]:
e = list(doc.sents)
x = e[1]
x.

''

In [37]:
[el for el in filter(lambda x: x.label_=="WORK_OF_ART", doc.ents)]

[The Martyrdom of the Ten Thousand (1508]

In [46]:
options = {'compact': True, #'bg': '#09a3d5',
           #'color': 'white', 
           'font': 'Source Sans Pro'}
displacy.render(doc, style='dep', options=options, jupyter=True)

In [26]:
doc.ents

(1507, Dürer, 
 Adam, Eve, 1507, The Martyrdom of the Ten Thousand, Frederick of Saxony, 
 Virgin, Iris, the Virgin (, 1509, Jacob Heller, Frankfurt, , the Trinity (1511, Matthaeus Landauer)

In [39]:
url = "https://en.wikipedia.org/w/api.php"
params = {"action": "parse",
          "format": "json"}
params["page"] = title
r = requests.get(url, params=params)

In [40]:
raw_text = r.json()["parse"]["text"]["*"]
soup = BeautifulSoup(raw_text, "html5lib")
text = soup.get_text()

In [61]:
title = "Albrecht_Dürer"
page = wiki_wiki.page(title)

In [144]:
def add_section(section=None, path=None, page=None, parent="", output={}):
    if isinstance(section, (WikipediaPage, WikipediaPageSection)):
        title = section.title
        text = section.text
        level = section.level
    else:  
        title = "summary"
        text = section
        level = 1
    new_path = f"{path}/{title}"
    start = page.index(text)
    end = start + len(text)
    hash_value = hashlib.md5(new_path.encode()).hexdigest()
    output[hash_value] = {"parent": parent, 
                          "level": level, 
                          "start": start,
                          "end": end,
                          "title": title,
                          "path": new_path}
    return output, new_path

In [142]:
def parse_sections(data, page=None, output=None, parent="", path=""):
    if output is None:
        output = {}
    if isinstance(data, WikipediaPage):
        output, _ = add_section(data.summary, path, page, parent, output=output)
    for s in data.sections:
        output, new_path = add_section(s, path, page, parent, output=output)
        if s.sections:
            parse_sections(s, parent=s.title, output=output, path=new_path, page=page)
    return output

In [145]:
d = parse_sections(page, page=page.text)

In [150]:
# for key, value in d.items():
#     print(value["title"])
#     print(value["start"], value["end"])
#     print(page.text[value["start"]:value["end"]])